In [ ]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.SILENT)

In [ ]:
model = LayeredNetwork('layeredNetwork')

# Create processors, tasks, and entries - client
P1 = Processor(model, 'P1', 1, SchedStrategy.INF)
T1 = Task(model, 'T1', 5, SchedStrategy.REF).on(P1)
E1 = Entry(model, 'E1').on(T1)

# Server 1
P2 = Processor(model, 'P2', 1, SchedStrategy.PS)
T2 = Task(model, 'T2', 1, SchedStrategy.FCFS).on(P2)
E2 = Entry(model, 'E2').on(T2)

# Server 2
P3 = Processor(model, 'P3', 1, SchedStrategy.PS)
T3 = Task(model, 'T3', 1, SchedStrategy.FCFS).on(P3)
E3 = Entry(model, 'E3').on(T3)

# DB server
PDB = Processor(model, 'PDB', 1, SchedStrategy.PS)
TDB = Task(model, 'TDB', 1, SchedStrategy.FCFS).on(PDB)
EDB = Entry(model, 'EDB').on(TDB)

# Cache task
totalitems = 2
cachecapacity = 1
# Create uniform access probabilities using Matrix
from jline.util.matrix import Matrix
access_probs = Matrix(1, totalitems).fill(1.0 / totalitems)
pAccess = DiscreteSampler(access_probs)
PC = Processor(model, 'PC', 1, SchedStrategy.PS)
C2 = CacheTask(model, 'C2', totalitems, cachecapacity, ReplacementStrategy.RR, 1).on(PC)
I2 = ItemEntry(model, 'I2', totalitems, pAccess).on(C2)

# Definition of activities

# Client layer
A1 = Activity(model, 'A1', Exp(2.0)).on(T1).boundTo(E1).synchCall(E2, 1)
A1end = Activity(model, 'A1end', Immediate()).on(T1).repliesTo(E1)

# Server layer for E2 (calling I2)
A2 = Activity(model, 'A2', Exp(1.0)).on(T2).boundTo(E2).synchCall(I2, 1)
A2end = Activity(model, 'A2end', Immediate()).on(T2).repliesTo(E2)

# Server layer for E3 (calling EDB)
A3 = Activity(model, 'A3', Exp(1.0)).on(T3).boundTo(E3).synchCall(EDB, 1)
A3end = Activity(model, 'A3end', Immediate()).on(T3).repliesTo(E3)

# Cache task layer
AC2 = Activity(model, 'AC2', Immediate()).on(C2).boundTo(I2)
AC2h = Activity(model, 'AC2h', Exp(1.0)).on(C2).repliesTo(I2)  # hit
AC2m = Activity(model, 'AC2m', Exp(0.5)).on(C2).synchCall(E3, 1)  # miss
AC2end = Activity(model, 'AC2end', Immediate()).on(C2).repliesTo(I2)

# Database layer
ADB = Activity(model, 'ADB', Exp(0.5)).on(TDB).boundTo(EDB).repliesTo(EDB)

# Add precedences
T1.addPrecedence(ActivityPrecedence.PRE(A1))
T1.addPrecedence(ActivityPrecedence.POST(A1end))
T2.addPrecedence(ActivityPrecedence.PRE_SEQ([A2, A2end]))
T3.addPrecedence(ActivityPrecedence.PRE_SEQ([A3, A3end]))
TDB.addPrecedence(ActivityPrecedence.PRE(ADB))
C2.addPrecedence(ActivityPrecedence.CacheAccess(AC2, [AC2h, AC2m]))
C2.addPrecedence(ActivityPrecedence.POST(AC2end))

In [1]:
lnoptions = SolverLN.defaultOptions()
lnoptions.verbose = 1
ncoptions = SolverNC.defaultOptions()
ncoptions.verbose = 0
mvaoptions = SolverMVA.defaultOptions()
mvaoptions.verbose = 0
solver1 = SolverLN(model, lambda model: SolverNC(model, ncoptions), lnoptions)
AvgTable[0] = solver1.get_avg_table()

NameError: name 'SolverLN' is not defined

In [ ]:
solver2 = SolverLN(model, lambda model: SolverMVA(model, mvaoptions), lnoptions)
AvgTable[1] = solver2.get_avg_table()